<a href="https://colab.research.google.com/github/auriwan/Recommendation-system/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive

In [2]:
!pip install -q kaggle

In [3]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [4]:
%cd /content/MyDrive/

[Errno 2] No such file or directory: '/content/MyDrive/'
/content


In [6]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/MyDrive/"

In [9]:
!chmod 600 /content/gdrive/MyDrive/kaggle.json

In [10]:
! KAGGLE_CONFIG_DIR=/content/gdrive/MyDrive kaggle datasets download -d CooperUnion/anime-recommendations-database

 36% 9.00M/25.0M [00:01<00:02, 7.03MB/s]
100% 25.0M/25.0M [00:01<00:00, 21.6MB/s]
